# Multi Class Classification with NLLLoss

In [8]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 --quiet

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

import time
import numpy as np
import matplotlib.pyplot as plt
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
train_data = './Dataset/train/'
test_data = './Dataset/test/'
device

'cpu'

In [12]:
!nvidia-smi

Sat Jul 23 11:59:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   39C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Custom Loader

In [63]:
img_size = 224

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
    transforms.Grayscale(),
    transforms.Resize((img_size,img_size))])


class CustomLoader(torch.utils.data.Dataset):
    def __init__(self, data_path, transform):
        self.path = data_path
        self.files = []
        for dir_path, currentDirectory, dir_files in os.walk(data_path):
            for file in dir_files:
                self.files.append(dir_path + '/' + file)
        self.indices = len(self.files)
        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.files[idx]).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        label = int(self.files[idx].split('-')[0][-1:])

        return image.to(device), torch.tensor(label).long().to(device)

    def __len__(self):
        return self.indices

In [64]:
data = CustomLoader(train_data, transform)
im, label = data[4850]
plt.imshow(im.permute(1,2,0).cpu())
print(label)

AssertionError: Torch not compiled with CUDA enabled

## CNN Model

In [56]:
!pip install torchsummary --quiet

In [65]:
def conv_layer(ni,no,kernel_size,stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, no, kernel_size, stride),
        nn.ReLU(),
        nn.BatchNorm2d(no),
        nn.MaxPool2d(2)
    )

def get_model():
    model = nn.Sequential(
        conv_layer(1, 64, 3),
        conv_layer(64, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        conv_layer(512, 512, 3),
        nn.Flatten(),
        nn.Linear(512, 4),
        nn.LogSoftmax(dim=1)
    ).to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)

    return model, loss_fn, optimizer

from torchsummary import summary
model, loss_fn, optimizer = get_model()
summary(model, (1, 224, 224));

AssertionError: Torch not compiled with CUDA enabled

## Load Train and Test

In [58]:
train_data_size = 5121
test_data_size = 1279

def get_data(train_data, test_data):
    train = CustomLoader(train_data, transform)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True, drop_last=True)

    test = CustomLoader(test_data, transform)
    test_dl = DataLoader(test, batch_size=32, shuffle=True, drop_last=True)

    return trn_dl, test_dl

trn_dl, test_dl = get_data(train_data, test_data)

## Train model

In [59]:
def train_and_validate(model, loss_criterion, optimizer, epochs=1):
    start = time.time()
    history = []
    best_loss = 100000.0
    best_epoch = None

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))

        # Set to training mode
        model.train()

        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0

        valid_loss = 0.0
        valid_acc = 0.0

        for i, (inputs, labels) in enumerate(trn_dl):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Clean existing gradients
            optimizer.zero_grad()

            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)

            # Compute loss
            loss = loss_criterion(outputs, labels)

            # Backpropagate the gradients
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)

            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))

            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))

            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)

            print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))


        # Validation - No gradient tracking needed
        with torch.no_grad():
            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(trn_dl):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch

        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/test_data_size
        avg_valid_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        epoch_end = time.time()

        print("Epoch : {:03d}, Training: Loss - {:.4f}, Accuracy - {:.4f}%, \n\t\tValidation : Loss - {:.4f}, Accuracy - {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))

        # Save if the model has best accuracy till now
        torch.save(model,'./model/model_'+str(epoch)+'.pt')

    return model, history, best_epoch

In [61]:
num_epochs = 6
model_real, loss_fn_real, optimizer_real = get_model()
trained_model_real, history_real, best_epoch = train_and_validate(model_real, loss_fn_real, optimizer_real, num_epochs)

Epoch: 1/6
Batch number: 000, Training: Loss: 2.3570, Accuracy: 0.0625
Batch number: 001, Training: Loss: 2.5487, Accuracy: 0.5625


KeyboardInterrupt: 

### Plot results

In [ ]:
torch.save(history_real, './model/classifier_real_images/history.pt')
history = np.array(history_real)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.savefig('./model/classifier_real_images/loss_curve.png')
plt.show()
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig('./model/classifier_real_images/accuracy_curve.png')
plt.show()

In [ ]:
torch.save(d_loss_history, './model/classifier_real_images/history_real.pt')

In [ ]:
data = CustomLoader(test_data, transform)
im, label = data[1001]
plt.imshow(im.permute(1,2,0).cpu())
label

In [ ]:
test = im.unsqueeze(0)
prediction = model_real(test).cpu().detach().numpy()
pred = np.exp(prediction)/np.sum(np.exp(prediction))
pred

In [ ]:
PATH = "./model/classifier_real_images/model.pt"

torch.save({
    'model': trained_model_real.state_dict(),
    'optimizer': optimizer_real.state_dict()
}, PATH)

## Train with GAN Images

In [ ]:
train_data = './gan_images/train/'
test_data = './gan_images/test/'

trn_dl, test_dl = get_data(train_data, test_data)

In [ ]:
num_epochs = 6
model_fake, loss_fn_fake, optimizer_fake = get_model()
trained_model_fake, history_fake, best_epoch = train_and_validate(model_fake, loss_fn_fake, optimizer_fake, num_epochs)

### Plot results

In [ ]:
torch.save(history_fake, './model/classifier_gan_images/history.pt')
history = np.array(history_fake)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.savefig('./model/classifier_gan_images/loss_curve.png')
plt.show()
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig('./model/classifier_gan_images/accuracy_curve.png')
plt.show()

In [ ]:
PATH = "./model/classifier_gan_images/model.pt"

torch.save({
    'model': trained_model_fake.state_dict(),
    'optimizer': optimizer_fake.state_dict()
}, PATH)

history_fake

In [ ]:
torch.save(d_loss_history, './model/classifier_gan_images/history_fake.pt')